<h1>Stock Market Prediction</h1>
After Stock market analysis, lets start with stock market prediction.
For Stock market Prediction we will be using different algorithms and models. In this notebook we will be trying to fit our data in different models.
Various models, toolkits and algorithms used in this notebook are K-Nearest Neighbours, FBProphet, Auto-Arima and LSTM.
In this Notebook we will be trying to predict the stock prices in upcoming days and a indicator that will show us on which days the stock prices will go up or down.

In [1]:
#for k-nearest neighbours
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV

from stocker import Stocker #for stocker toolkit

from pyramid.arima import auto_arima #for ARIMA

#for LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#for importing stock data files
import file_import
#for Dataframe
import pandas as pd
#for Date and Time objects
import datetime
#for spliting the data into testing and training dataset
from sklearn.model_selection import train_test_split
#Matplotlib for ploting graphs
import matplotlib.pyplot as plt
import numpy as np
import call_api

/home/yuvraj/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/home/yuvraj/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

/home/yuvraj/.local/lib/python3.6/site-packages/pyramid/__init__.py:68: UserWarning:


    The 'pyramid' package will be migrating to a new namespace beginning in 
    version 1.0.0: 'pmdarima'. This is due to a package name collision with the
    Pyramid web framework. For more information

Starting with importing stock data into a Pandas Dataframe object

In [2]:
data, lis, file_name = file_import.operate_file() #Here data holds the stock data
#up_data = call_api.data_update(data, file_name, lis[0])
#data = pd.concat([data, up_data], sort = True)
stock_name = lis[0]
start_date = lis[1]
end_date = lis[2]
#data = data.drop(['1. open','2. high','3. low','4. close','5. volume','OpenInt'], axis = 1)
print("Lets us see the first 10 rows of the %s stock\n\n"%(stock_name),data)

Lets us see the first 10 rows of the wmt stock

                  Open       High        Low      Close      Volume  OpenInt
Date                                                                       
1972-03-20    0.03403    0.03403    0.03403    0.03403   3132119.0      0.0
1972-03-22    0.03403    0.04251    0.03403    0.03403   1084194.0      0.0
1972-03-23    0.03403    0.04251    0.03403    0.03403    783030.0      0.0
1972-03-24    0.03403    0.04251    0.03403    0.03403   1264892.0      0.0
1972-03-27    0.04251    0.04251    0.04251    0.04251   1385361.0      0.0
...               ...        ...        ...        ...         ...      ...
2019-11-14  124.60000  125.38000  119.51000  120.65000  22504200.0      NaN
2019-11-15  120.68000  121.00000  118.38000  118.87000   9531474.0      NaN
2019-11-18  118.45000  120.86700  118.24000  120.36500   2116593.0      NaN
2019-11-19  120.11000  120.36000  119.71000  119.89000   3715400.0      NaN
2019-11-20  120.21000  120.48000  119.3

In [3]:
data.drop_duplicates( keep = False, inplace = True) 
x = 0
while(x == 0):
    y,m,d = start_date.split('-',3)
    temp_date = datetime.date(int(y),int(m),int(d))
    
    df = data.loc[start_date : end_date, : ] #Slicing the data from start date to end date
    if(len(df.index) == 0):
        print("Date Interval specified is Invalid \nPlease, Re-Enter the date in Valid format YYYY-MM-DD : \n")
        continue
    x = 1

df.index = pd.to_datetime(df.index) #Converting index to datetime object so that daily, monthly or yearly data could be taken out

In [4]:
import indexers
indexer = indexers()
df = indexer.operate_file(df, str(start_date), str(end_date))

IndentationError: expected an indented block (indexers.py, line 22)

After importing stock data file into a DataFrame object let us split that data into training and testing dataframes.

In [ ]:
li = df.shape
x = int(li[0] - 0.2*li[0])
train = df[ : x]
valid = df[x : ]
x_train = train.drop('Close', axis=1)
y_train = train['Close']
x_valid = valid.drop('Close', axis=1)
y_valid = valid['Close']
len_valid = valid.shape[0]

After splitting the data, Let us now apply machine learning algorithms and model.
<h2>K-Nearest Neighbours</h2>
One of the most interesting ML algorithm that one can use here is kNN (k nearest neighbours). Based on the independent variables, kNN finds the similarity between new data points and old data points.
<h3>Pseudo Code of KNN</h3>
We can implement a KNN model by following the below steps:
<ol>
    <li>Load the data</li>
    <li>Initialise the value of k</li>
    <li>For getting the predicted class, iterate from 1 to total number of training data points</li><ol>
        <li>Calculate the distance between test data and each row of training data. Here we will use Euclidean distance as our distance metric since it’s the most popular method. The other metrics that can be used are Chebyshev, cosine, etc.</li>
        <li>Sort the calculated distances in ascending order based on distance values</li>
        <li>Get top k rows from the sorted array</li>
        <li>Get the most frequent class of these rows</li>
        <li>Return the predicted class</li></ol>
</ol>

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
#scaling data
x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)
x_valid_scaled = scaler.fit_transform(x_valid)
x_valid = pd.DataFrame(x_valid_scaled)

#using gridsearch to find the best parameter
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)

#fit the model and make predictions
model.fit(x_train,y_train)
preds = model.predict(x_valid)

<h3>Results</h3>

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(y_valid)-np.array(preds)),2)))
print(str(rms) + "\n\n\n")

In [ ]:
if('Prediction' in valid.columns):
    valid.drop('Prediction', axis = 1)
    
#ploting using matplotlib
valid['Predictions'] = preds
%matplotlib inline
%pylab inline

pylab.rcParams['figure.figsize'] = (15, 9)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.xlabel('Time---->')
plt.ylabel('Price in Dollars----->')
plt.legend()
plt.grid(True)

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(y_valid)-np.array(preds)),2)))
print(str(rms) + "\n\n\n")

In [ ]:
import simple_scaling
valid = simple_scaling.simple(data = valid)

In [ ]:
%matplotlib inline
%pylab inline

pylab.rcParams['figure.figsize'] = (15, 9)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions','New Predictions']])
plt.xlabel('Time---->')
plt.ylabel('Price in Dollars----->')
plt.legend()
plt.grid(True)

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(valid['Close'])-np.array(valid['New Predictions'])),2)))
print(str(rms) + "\n\n\n")

In [ ]:
valid_new = simple_scaling.additive(valid)
plt.plot(train['Close'])
plt.plot(valid_new[['Close','Predictions','New Prediction']])
plt.xlabel('Time---->')
plt.ylabel('Price in Dollars----->')
plt.grid(True)

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(valid_new["Close"])-np.array(valid_new["New Prediction"])),2)))
print(str(rms) + "\n\n\n")

<h2>Auto ARIMA</h2>
Introduction

ARIMA is a very popular statistical method for time series forecasting. ARIMA models take into account the past values to predict the future values. There are three important parameters in ARIMA:

    p (past values used for forecasting the next value)
    q (past forecast errors used to predict the future values)
    d (order of differencing)

Parameter tuning for ARIMA consumes a lot of time. So we will use auto ARIMA which automatically selects the best combination of (p,q,d) that provides the least error. 

In [ ]:
training = train['Close']
validation = valid['Close']

model = auto_arima(training, start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0, seasonal=True,d=1, D=1, trace=True,error_action='ignore',suppress_warnings=True)
model.fit(training)

forecast = model.predict(n_periods=len_valid)
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])


<h3>Results</h3>

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(valid['Close'])-np.array(forecast['Prediction'])),2)))
rms

In [ ]:
#plot
plt.plot(train['Close'])
plt.plot(valid['Close'])
plt.plot(forecast['Prediction'])
plt.xlabel('Time---->')
plt.ylabel('Price in Dollars----->')
plt.show()

<h2>Long Short Term Memory (LSTM)</h2>

LSTMs are widely used for sequence prediction problems and have proven to be extremely effective. The reason they work so well is because LSTM is able to store past information that is important, and forget the information that is not. LSTM has three gates:

    The input gate: The input gate adds information to the cell state
    The forget gate: It removes the information that is no longer required by the model
    The output gate: Output Gate at LSTM selects the information to be shown as output


In [ ]:
#creating dataframe
df = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(df)):
    new_data['Date'][i] = df.index[i]
    new_data['Close'][i] = df['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

li = dataset.shape
x = int(li[0] - 0.2*li[0])
train = dataset[ : x]
valid = dataset[x : ]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=3, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
li = new_data.shape
x = int(li[0] - 0.2*li[0])
train = new_data[ : x]
valid = new_data[x : ]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.grid(False)

In [ ]:
#creating dataframe
df = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(df)):
    new_data['Date'][i] = df.index[i]
    new_data['Close'][i] = df['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

li = dataset.shape
x = int(li[0] - 0.2*li[0])
train = dataset[ : x]
valid = dataset[x : ]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=4, batch_size=4, verbose=4)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
li = new_data.shape
x = int(li[0] - 0.2*li[0])
train = new_data[ : x]
valid = new_data[x : ]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.grid(False)

<h2>Begin with Stocker</h2>

In [ ]:
stock = Stocker(stock_name, df)

In [ ]:
stock.plot_stock(stats=['Daily Change'])

In [ ]:
model, model_data = stock.create_prophet_model()

In [ ]:
model.plot_components(model_data)
plt.show()

In [ ]:
model, model_data = stock.create_prophet_model(days=90)

In [ ]:
stock.evaluate_prediction()

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>